In [31]:
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.ndimage import gaussian_filter
from tifffile.tifffile import imwrite

import caiman as cm
from caiman.utils.visualization import nb_view_patches3d
import caiman.source_extraction.cnmf as cnmf
from caiman.paths import caiman_datadir

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import bokeh.plotting as bpl
bpl.output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [32]:

logfile = None # Replace with a path if you want to log to a file
logger = logging.getLogger('caiman')
# Set to logging.INFO if you want much output, potentially much more output
logger.setLevel(logging.WARNING)
logfmt = logging.Formatter('%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s')
if logfile is not None:
    handler = logging.FileHandler(logfile)
else:
    handler = logging.StreamHandler()
handler.setFormatter(logfmt)
logger.addHandler(handler)

In [57]:
raw_data_path = r'/mnt/md0/runs/s2p-SS003_2024-08-06_1-2-3-4-5-6/raw'

In [58]:
tifs = [os.path.join(raw_data_path, t) for t in np.sort(os.listdir(raw_data_path))]

In [65]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)

In [66]:
%%time
# motion correction parameters
opts_dict = {'fnames': tifs,
            'strides': (6,24, 24),    # start a new patch for pw-rigid motion correction every x pixels
            'overlaps': (2,12, 12),   # overlap between patches (size of patch strides+overlaps)
            'max_shifts': (5,30, 30, ),   # maximum allowed rigid shifts (in pixels)
            'max_deviation_rigid': 5,  # maximum shifts deviation allowed for patch with respect to rigid shifts
            'pw_rigid': False,         # flag for performing non-rigid motion correction
            'is3D': True}

opts = cnmf.params.CNMFParams(params_dict=opts_dict)
mc = cm.motion_correction.MotionCorrect(tifs, dview=dview, **opts.get_group('motion'))

   114324110 [params.py:       change_params():1156] [119995] In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
   114324110 [params.py:       change_params():1156] [119995] In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
   114324641 [params.py:   check_consistency():964] [119995] is3D=True, hence setting key indices to (slice(None, None, None), slice(None, None, None), slice(None, None, None))
   114324641 [params.py:   check_consistency():964] [119995] is3D=True, hence setting key indices to (slice(None, None, None), slice(None, None, None), slice(None, None, None))


CPU times: user 476 ms, sys: 55.9 ms, total: 532 ms
Wall time: 533 ms


In [67]:
# for 10 tifs:
# 1h2min motion corr 
# 20min + 1h40 min + 1min for detection

# for 5 tifs
# 29min motion corr
# 20 minutes + 1h25 + 30sec

# for 192 tifs:


In [ ]:
%%time
%%capture
#%% Run motion correction using NoRMCorre
tic = time.time()
mc.motion_correct(save_movie=True)
toc = time.time()
mc_time = toc - tic
print(mc_time)

In [ ]:
mc_time

In [ ]:
%%time
tic = time.time()
#%% MEMORY MAPPING
# memory map the file in order 'C'
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=0, dview=dview) # exclude borders

# now load the file
Yr, dims, T = cm.load_memmap(fname_new)
images = np.reshape(Yr.T, [T] + list(dims), order='F') 
    #load frames in python format (T x X x Y)
mmap_time = time.time() - tic
print(mmap_time)

In [ ]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)

In [ ]:
# set parameters
rf = 15  # half-size of the patches in pixels. rf=25, patches are 50x50
stride = 10  # amount of overlap between the patches in pixels
K = 10  # number of neurons expected per patch
gSig = [1,2,2]  # expected half size of neurons
merge_thresh = 0.8  # merging threshold, max correlation allowed
p = 2  # order of the autoregressive system
print('set')

In [ ]:
%%time
tic = time.time()
cnm = cnmf.CNMF(n_processes, 
                k=K, 
                gSig=gSig, 
                merge_thresh=merge_thresh, 
                p=p, 
                dview=dview,
                rf=rf, 
                stride=stride, 
                only_init_patch=True)
cnm.params.set('spatial', {'se': np.ones((3,3,1), dtype=np.uint8)})
cnm = cnm.fit(images)
print(('Number of components:' + str(cnm.estimates.A.shape[-1])))
cnmf_time = time.time() - tic
print(cnmf_time)

In [ ]:
%%time
tic = time.time()
#%% COMPONENT EVALUATION
# the components are evaluated in two ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient

fr = 5 # approx final rate  (after eventual downsampling )
decay_time = 1.5  # length of typical transient in seconds 
use_cnn = False  # CNN classifier is designed for 2d (real) data
min_SNR = 3      # accept components with that peak-SNR or higher
rval_thr = 0.6   # accept components with space correlation threshold or higher
cnm.params.change_params(params_dict={'fr': fr,
                                      'decay_time': decay_time,
                                      'min_SNR': min_SNR,
                                      'rval_thr': rval_thr,
                                      'use_cnn': use_cnn})

cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
eval_time = time.time() - tic
print(eval_time)

In [ ]:
print(('Keeping ' + str(len(cnm.estimates.idx_components)) +
       ' and discarding  ' + str(len(cnm.estimates.idx_components_bad))))

In [ ]:
%%time
cnm.params.set('temporal', {'p': p})
cnm2 = cnm.refit(images)

In [ ]:
import time

In [ ]:
end_time = time.time()

In [ ]:
print(end_time)

In [ ]:
# 1742597038.8514998


In [ ]:
# start_time is 19:32 on 2025-03-21

In [ ]:
import datetime
datetime.datetime.fromtimestamp(end_time).strftime('%c')

In [29]:
Yr.shape

(12330000, 1000)